In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
path_trainFile = "train_preliminary"
path_testFile = "test"

In [3]:
path_cl_tr = os.path.join(path_trainFile, "click_log.csv")
path_cl_te = os.path.join(path_testFile, "click_log.csv")

In [4]:
df_cl_tr = pd.read_csv(path_cl_tr, dtype={'creative_id': str})
df_cl_te = pd.read_csv(path_cl_te, dtype={'creative_id': str})

In [5]:
path_user_tr = os.path.join(path_trainFile, "user.csv")

In [6]:
df_user_tr = pd.read_csv(path_user_tr)

In [7]:
df_cl_tr.shape, df_cl_te.shape

((30082771, 4), (33585512, 4))

In [8]:
df_cl = pd.concat([df_cl_tr, df_cl_te], axis=0)

In [9]:
df_cl.shape

(63668283, 4)

In [10]:
len(df_cl['user_id'].unique()), max(df_cl['user_id']), min(df_cl['user_id'])

(1900000, 4000000, 1)

In [11]:
len(df_cl['creative_id'].unique()), max(df_cl['creative_id']), min(df_cl['creative_id'])

(3412772, '999998', '1')

In [ ]:
3132000

In [12]:
sentences = df_cl.groupby(['user_id']).apply(lambda x: x['creative_id'].tolist()).tolist()

In [13]:
from gensim.models import Word2Vec

In [14]:
model = Word2Vec(sentences=sentences, min_count=1, sg=1, window=10, size=32, workers=22, seed=2020, iter=5)

In [15]:
user_feature_tr = df_cl_tr.groupby(['user_id']).apply(lambda x: np.mean([model[v] for v in x['creative_id'].tolist()], axis=0))

/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [16]:
user_feature_tr = user_feature_tr.reset_index()

In [17]:
user_feature_tr.columns = ['user_id', 'feature']

In [18]:
user_feature_tr.head()

,user_id,feature
0,1,"[-0.1665218, 0.079023324, -0.4205783, -0.03808..."
1,2,"[0.13393308, -0.127439, -0.5133559, -0.4447803..."
2,3,"[-0.37463346, -0.023476254, -0.95639664, -0.12..."
3,4,"[-0.06618744, 0.23780581, -0.8613778, -0.11283..."
4,5,"[0.09752068, -0.090954155, -0.24275202, -0.082..."


In [41]:
len(user_feature_tr['user_id'].unique()), min(user_feature_tr['user_id']), max(user_feature_tr['user_id'])

(900000, 1, 900000)

In [19]:
tr_valid = pd.merge(df_user_tr, user_feature_tr, on='user_id')

In [20]:
tr_valid.head()

,user_id,age,gender,feature
0,1,4,1,"[-0.1665218, 0.079023324, -0.4205783, -0.03808..."
1,2,10,1,"[0.13393308, -0.127439, -0.5133559, -0.4447803..."
2,3,7,2,"[-0.37463346, -0.023476254, -0.95639664, -0.12..."
3,4,5,1,"[-0.06618744, 0.23780581, -0.8613778, -0.11283..."
4,5,4,1,"[0.09752068, -0.090954155, -0.24275202, -0.082..."


In [21]:
from sklearn.model_selection  import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(np.array(tr_valid['feature'].tolist()), tr_valid[['age', 'gender']], test_size=0.33, random_state=42)

In [24]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [25]:
model_age = XGBClassifier(n_jobs=20)
model_gender = XGBClassifier(n_jobs=20)

In [26]:
model_age.fit(X_train,y_train['age'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=20, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [27]:
model_gender.fit(X_train,y_train['gender'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=20, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [56]:
X_test

array([[-0.44787315,  0.1145662 , -0.28071478, ...,  0.14111398,
         0.12033978,  0.07984406],
       [-0.26163363, -0.03761578, -0.43959522, ...,  0.26254553,
        -0.25736097,  0.33707008],
       [-0.11975394, -0.06346972, -0.6211729 , ...,  0.02860467,
        -0.03169598,  0.13268326],
       ...,
       [-0.07474452,  0.38196114, -0.35133412, ...,  0.5400635 ,
         0.3173578 , -0.31480545],
       [ 0.02563123, -0.2706324 , -0.75557953, ...,  0.34179646,
         0.2334501 ,  0.17892851],
       [-0.44029504,  0.07397053, -0.8503374 , ...,  0.43267587,
        -0.20881107,  0.1232922 ]], dtype=float32)

In [28]:
y_pred = model_age.predict(X_test)

In [29]:
accuracy = accuracy_score(y_test['age'], y_pred)

In [30]:
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 35.30%


In [31]:
y_pred = model_gender.predict(X_test)

In [32]:
accuracy = accuracy_score(y_test['gender'], y_pred)

In [33]:
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 91.74%


In [42]:
user_feature_te = df_cl_te.groupby(['user_id']).apply(lambda x: np.mean([model[v] for v in x['creative_id'].tolist()], axis=0))

/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [43]:
user_feature_te = user_feature_te.reset_index()

In [44]:
user_feature_te.columns = ['user_id', 'feature']

In [51]:
user_feature_te['feature'].head()

0    [-0.22963852, 0.017190743, 0.015032875, -0.252...
1    [-0.0730542, 0.02119232, -0.61678326, -0.18522...
2    [0.10934746, -0.09705918, -0.33597443, -0.3008...
3    [0.3220719, 0.04133042, -0.63475734, -0.545194...
4    [0.009124441, 0.40578339, -0.5639814, -0.51051...
Name: feature, dtype: object

In [65]:
user_feature_te['predicted_age'] = model_age.predict(np.array(user_feature_te['feature'].tolist()))

In [66]:
user_feature_te['predicted_gender'] = model_gender.predict(np.array(user_feature_te['feature'].tolist()))

In [68]:
user_feature_te[['user_id', 'predicted_age', 'predicted_gender']].to_csv('submission.csv', index=False, encoding='utf-8')

In [70]:
from ti import session
ti_session = session.Session()
inputs = ti_session.upload_data(path="submission.csv", bucket="lbo00000-1259553731", key_prefix="tencent_ad_result/0604")